# Waymo Open Dataset Explorer

This notebook provides comprehensive visualization tools for exploring the Waymo Open Dataset.

## Features:
- Load and explore trajectory data
- Visualize vehicle trajectories (history and future)
- Analyze social interactions between vehicles
- Statistical analysis of driving patterns
- Interactive visualizations

## 1. Setup and Imports

In [ ]:
import sys
import os
sys.path.append('..')  # Add parent directory to path

import numpy as np
import torch
import yaml
from pathlib import Path
import pickle
from tqdm import tqdm

# Visualization libraries
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import HTML, display
import ipywidgets as widgets

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Import our custom modules
from data.waymo_loader import WaymoTrajectoryDataset
from models.csp_model import ConvSocialPoolingModel

print("✅ All imports successful!")

## 2. Load Configuration and Dataset

In [ ]:
# Load configuration
config_path = Path('../configs/default_config.yaml')
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

print("📋 Configuration loaded:")
print(f"  History: {config['data']['history_seconds']}s ({int(config['data']['history_seconds'] * config['data']['sample_rate'])} frames)")
print(f"  Future: {config['data']['future_seconds']}s ({int(config['data']['future_seconds'] * config['data']['sample_rate'])} frames)")
print(f"  Sample rate: {config['data']['sample_rate']} Hz")
print(f"  Social radius: {config['data']['social_radius']} meters")
print(f"  Max neighbors: {config['data']['max_neighbors']}")

In [ ]:
# Initialize dataset
print("\n🚗 Loading Waymo dataset...")
dataset = WaymoTrajectoryDataset(
    data_path=config['data']['waymo_path'],
    split='training',
    config=config,
    cache_dir='../cache'
)

print(f"\n✅ Dataset loaded successfully!")
print(f"  Total samples: {len(dataset):,}")
print(f"  History length: {dataset.history_len} frames")
print(f"  Future length: {dataset.future_len} frames")

## 3. Data Exploration

In [ ]:
# Get a sample from the dataset
sample_idx = 0
sample = dataset[sample_idx]

print("📊 Sample data structure:")
for key, value in sample.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: shape {value.shape}, dtype {value.dtype}")
    else:
        print(f"  {key}: {type(value)}")

# Display sample values
print("\n🔍 Sample trajectory (first 5 frames):")
print("  History:")
print(f"    x, y, vx, vy, heading")
for i in range(min(5, len(sample['history']))):
    h = sample['history'][i]
    print(f"    [{h[0]:7.2f}, {h[1]:7.2f}, {h[2]:6.2f}, {h[3]:6.2f}, {h[4]:5.2f}]")

## 4. Trajectory Visualization Functions

In [ ]:
def plot_trajectory_2d(history, future, title="Vehicle Trajectory", ax=None, 
                       history_color='blue', future_color='green', 
                       show_velocity=False, show_direction=True):
    """Plot a single vehicle trajectory in 2D."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 10))
    
    # Convert tensors to numpy if needed
    if isinstance(history, torch.Tensor):
        history = history.numpy()
    if isinstance(future, torch.Tensor):
        future = future.numpy()
    
    # Plot history trajectory
    ax.plot(history[:, 0], history[:, 1], 'o-', color=history_color, 
            label='History', linewidth=2, markersize=4, alpha=0.7)
    
    # Plot future trajectory
    ax.plot(future[:, 0], future[:, 1], 's-', color=future_color, 
            label='Future', linewidth=2, markersize=4, alpha=0.7)
    
    # Connect history and future
    ax.plot([history[-1, 0], future[0, 0]], [history[-1, 1], future[0, 1]], 
            'k--', alpha=0.3, linewidth=1)
    
    # Mark start and end points
    ax.scatter(history[0, 0], history[0, 1], c='red', s=100, 
              marker='^', label='Start', zorder=5)
    ax.scatter(future[-1, 0], future[-1, 1], c='darkred', s=100, 
              marker='v', label='End', zorder=5)
    
    # Show velocity vectors if requested
    if show_velocity:
        # Sample every 5th point for clarity
        for i in range(0, len(history), 5):
            ax.arrow(history[i, 0], history[i, 1], 
                    history[i, 2]*2, history[i, 3]*2,
                    head_width=0.5, head_length=0.3, 
                    fc=history_color, ec=history_color, alpha=0.5)
    
    # Show direction arrows
    if show_direction:
        # Add arrow showing direction at current position
        if len(history) > 0:
            heading = history[-1, 4] if history.shape[1] > 4 else 0
            arrow_len = 5
            ax.arrow(history[-1, 0], history[-1, 1],
                    arrow_len * np.cos(heading), arrow_len * np.sin(heading),
                    head_width=2, head_length=1.5, fc='orange', ec='orange',
                    label='Current heading', zorder=4)
    
    ax.set_xlabel('X Position (meters)', fontsize=12)
    ax.set_ylabel('Y Position (meters)', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal', adjustable='box')
    
    return ax

In [ ]:
# Visualize a single trajectory
sample = dataset[0]
plot_trajectory_2d(sample['history'], sample['future'], 
                  title="Sample Vehicle Trajectory")
plt.tight_layout()
plt.show()

## 5. Social Context Visualization

In [ ]:
def visualize_social_tensor(social_tensor, ax=None):
    """Visualize the social pooling tensor as a grid."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(12, 8))
    
    if isinstance(social_tensor, torch.Tensor):
        social_tensor = social_tensor.numpy()
    
    # Extract occupancy grid (first channel)
    occupancy = social_tensor[:, :, 0]
    
    # Create heatmap
    im = ax.imshow(occupancy.T, cmap='YlOrRd', aspect='auto', origin='lower')
    
    # Add grid
    for i in range(occupancy.shape[0] + 1):
        ax.axvline(i - 0.5, color='gray', linewidth=0.5, alpha=0.5)
    for i in range(occupancy.shape[1] + 1):
        ax.axhline(i - 0.5, color='gray', linewidth=0.5, alpha=0.5)
    
    # Add annotations for occupied cells
    for i in range(occupancy.shape[0]):
        for j in range(occupancy.shape[1]):
            if occupancy[i, j] > 0:
                # Get velocity for this cell
                vx = social_tensor[i, j, 3]
                vy = social_tensor[i, j, 4]
                speed = np.sqrt(vx**2 + vy**2)
                ax.text(i, j, f'{speed:.1f}', ha='center', va='center',
                       color='white' if occupancy[i, j] > 0.5 else 'black',
                       fontsize=8)
    
    ax.set_xlabel('Grid X (relative position)', fontsize=12)
    ax.set_ylabel('Grid Y (relative position)', fontsize=12)
    ax.set_title('Social Context Grid\n(Numbers show neighbor speeds)', fontsize=14)
    
    # Add colorbar
    plt.colorbar(im, ax=ax, label='Occupancy')
    
    # Mark ego vehicle position (center)
    center_x, center_y = occupancy.shape[0] // 2, occupancy.shape[1] // 2
    circle = patches.Circle((center_x, center_y), 0.5, linewidth=2, 
                           edgecolor='blue', facecolor='none', label='Ego vehicle')
    ax.add_patch(circle)
    
    ax.legend(loc='upper right')
    return ax

In [ ]:
# Visualize social context for a sample
sample = dataset[0]
visualize_social_tensor(sample['social_tensor'])
plt.tight_layout()
plt.show()

## 6. Multi-Vehicle Scene Visualization

In [ ]:
def plot_multi_vehicle_scene(dataset, sample_idx, num_neighbors=5):
    """Plot multiple vehicle trajectories in the same scene."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    
    # Get main vehicle trajectory
    sample = dataset[sample_idx]
    history = sample['history'].numpy() if isinstance(sample['history'], torch.Tensor) else sample['history']
    future = sample['future'].numpy() if isinstance(sample['future'], torch.Tensor) else sample['future']
    
    # Plot ego vehicle
    ax1.plot(history[:, 0], history[:, 1], 'b-', linewidth=3, label='Ego History', alpha=0.8)
    ax1.plot(future[:, 0], future[:, 1], 'g-', linewidth=3, label='Ego Future', alpha=0.8)
    
    # Get raw trajectory data to access neighbors
    if hasattr(dataset, 'trajectories') and sample_idx < len(dataset.trajectories):
        raw_sample = dataset.trajectories[sample_idx]
        if 'neighbors' in raw_sample:
            neighbors = raw_sample['neighbors'][:num_neighbors]
            
            # Plot neighbor positions
            ego_pos = history[-1, :2]
            for i, neighbor in enumerate(neighbors):
                neighbor_pos = ego_pos + np.array([neighbor['relative_x'], neighbor['relative_y']])
                ax1.scatter(neighbor_pos[0], neighbor_pos[1], s=100, 
                          label=f'Neighbor {i+1}', alpha=0.7)
                
                # Draw velocity vector
                ax1.arrow(neighbor_pos[0], neighbor_pos[1],
                         neighbor['velocity_x']*2, neighbor['velocity_y']*2,
                         head_width=1, head_length=0.5, alpha=0.5)
    
    ax1.set_xlabel('X Position (meters)')
    ax1.set_ylabel('Y Position (meters)')
    ax1.set_title('Multi-Vehicle Scene (World Frame)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_aspect('equal')
    
    # Plot in ego-centric frame
    ego_pos = history[-1, :2]
    history_rel = history.copy()
    future_rel = future.copy()
    history_rel[:, :2] -= ego_pos
    future_rel[:, :2] -= ego_pos
    
    ax2.plot(history_rel[:, 0], history_rel[:, 1], 'b-', linewidth=3, label='Ego History', alpha=0.8)
    ax2.plot(future_rel[:, 0], future_rel[:, 1], 'g-', linewidth=3, label='Ego Future', alpha=0.8)
    ax2.scatter(0, 0, s=200, c='red', marker='*', label='Current Position', zorder=5)
    
    # Add social radius circle
    circle = patches.Circle((0, 0), dataset.social_radius, linewidth=2,
                           edgecolor='orange', facecolor='none', 
                           linestyle='--', label=f'Social Radius ({dataset.social_radius}m)')
    ax2.add_patch(circle)
    
    ax2.set_xlabel('Relative X Position (meters)')
    ax2.set_ylabel('Relative Y Position (meters)')
    ax2.set_title('Multi-Vehicle Scene (Ego-Centric Frame)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_aspect('equal')
    
    plt.suptitle(f'Sample {sample_idx}: Multi-Vehicle Interaction', fontsize=16, fontweight='bold')
    return fig

In [ ]:
# Visualize multi-vehicle scene
fig = plot_multi_vehicle_scene(dataset, sample_idx=0)
plt.tight_layout()
plt.show()

## 7. Statistical Analysis

In [ ]:
def analyze_dataset_statistics(dataset, num_samples=1000):
    """Analyze statistical properties of the dataset."""
    velocities = []
    accelerations = []
    trajectory_lengths = []
    turn_angles = []
    
    num_samples = min(num_samples, len(dataset))
    
    for i in tqdm(range(num_samples), desc="Analyzing samples"):
        sample = dataset[i]
        history = sample['history'].numpy() if isinstance(sample['history'], torch.Tensor) else sample['history']
        
        # Calculate velocities
        vx, vy = history[:, 2], history[:, 3]
        speeds = np.sqrt(vx**2 + vy**2)
        velocities.extend(speeds)
        
        # Calculate accelerations
        if len(speeds) > 1:
            accel = np.diff(speeds) * dataset.config['data']['sample_rate']
            accelerations.extend(accel)
        
        # Calculate trajectory length
        positions = history[:, :2]
        if len(positions) > 1:
            dists = np.sqrt(np.sum(np.diff(positions, axis=0)**2, axis=1))
            trajectory_lengths.append(np.sum(dists))
        
        # Calculate turn angles
        if history.shape[1] > 4 and len(history) > 1:
            headings = history[:, 4]
            heading_changes = np.diff(headings)
            # Normalize to [-pi, pi]
            heading_changes = np.arctan2(np.sin(heading_changes), np.cos(heading_changes))
            turn_angles.extend(heading_changes)
    
    return {
        'velocities': np.array(velocities),
        'accelerations': np.array(accelerations),
        'trajectory_lengths': np.array(trajectory_lengths),
        'turn_angles': np.array(turn_angles)
    }

In [ ]:
# Analyze dataset statistics
print("📊 Analyzing dataset statistics...")
stats = analyze_dataset_statistics(dataset, num_samples=min(500, len(dataset)))

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Velocity distribution
axes[0, 0].hist(stats['velocities'], bins=50, color='blue', alpha=0.7, edgecolor='black')
axes[0, 0].axvline(np.mean(stats['velocities']), color='red', linestyle='--', 
                   label=f'Mean: {np.mean(stats["velocities"]):.2f} m/s')
axes[0, 0].set_xlabel('Velocity (m/s)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Velocity Distribution')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Acceleration distribution
axes[0, 1].hist(stats['accelerations'], bins=50, color='green', alpha=0.7, edgecolor='black')
axes[0, 1].axvline(np.mean(stats['accelerations']), color='red', linestyle='--',
                   label=f'Mean: {np.mean(stats["accelerations"]):.2f} m/s²')
axes[0, 1].set_xlabel('Acceleration (m/s²)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Acceleration Distribution')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Trajectory length distribution
axes[1, 0].hist(stats['trajectory_lengths'], bins=50, color='orange', alpha=0.7, edgecolor='black')
axes[1, 0].axvline(np.mean(stats['trajectory_lengths']), color='red', linestyle='--',
                   label=f'Mean: {np.mean(stats["trajectory_lengths"]):.2f} m')
axes[1, 0].set_xlabel('Trajectory Length (m)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Trajectory Length Distribution')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Turn angle distribution
axes[1, 1].hist(np.degrees(stats['turn_angles']), bins=50, color='purple', alpha=0.7, edgecolor='black')
axes[1, 1].axvline(np.degrees(np.mean(stats['turn_angles'])), color='red', linestyle='--',
                   label=f'Mean: {np.degrees(np.mean(stats["turn_angles"])):.2f}°')
axes[1, 1].set_xlabel('Turn Angle (degrees/frame)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Turn Angle Distribution')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('Waymo Dataset Statistical Analysis', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Print summary statistics
print("\n📈 Summary Statistics:")
print(f"  Velocity: {np.mean(stats['velocities']):.2f} ± {np.std(stats['velocities']):.2f} m/s")
print(f"  Acceleration: {np.mean(stats['accelerations']):.2f} ± {np.std(stats['accelerations']):.2f} m/s²")
print(f"  Trajectory Length: {np.mean(stats['trajectory_lengths']):.2f} ± {np.std(stats['trajectory_lengths']):.2f} m")
print(f"  Turn Angle: {np.degrees(np.mean(stats['turn_angles'])):.2f} ± {np.degrees(np.std(stats['turn_angles'])):.2f} degrees")

## 8. Interactive Trajectory Explorer

In [ ]:
def create_interactive_explorer(dataset, max_samples=100):
    """Create an interactive widget to explore different samples."""
    max_samples = min(max_samples, len(dataset))
    
    def plot_sample(sample_idx, show_velocity, show_social):
        sample = dataset[sample_idx]
        
        if show_social:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
            plot_trajectory_2d(sample['history'], sample['future'], 
                             title=f"Sample {sample_idx}: Vehicle Trajectory",
                             ax=ax1, show_velocity=show_velocity)
            visualize_social_tensor(sample['social_tensor'], ax=ax2)
        else:
            fig, ax = plt.subplots(figsize=(10, 10))
            plot_trajectory_2d(sample['history'], sample['future'],
                             title=f"Sample {sample_idx}: Vehicle Trajectory",
                             ax=ax, show_velocity=show_velocity)
        
        plt.tight_layout()
        plt.show()
    
    # Create widgets
    sample_slider = widgets.IntSlider(
        value=0, min=0, max=max_samples-1, step=1,
        description='Sample:', continuous_update=False
    )
    
    velocity_checkbox = widgets.Checkbox(
        value=False, description='Show Velocity Vectors'
    )
    
    social_checkbox = widgets.Checkbox(
        value=True, description='Show Social Context'
    )
    
    # Create interactive plot
    widgets.interact(plot_sample, 
                    sample_idx=sample_slider,
                    show_velocity=velocity_checkbox,
                    show_social=social_checkbox)

# Create the interactive explorer
print("🎮 Interactive Trajectory Explorer")
print("Use the controls below to explore different samples:")
create_interactive_explorer(dataset)

## 9. 3D Interactive Visualization with Plotly

In [ ]:
def create_3d_trajectory_plot(dataset, sample_idx):
    """Create an interactive 3D trajectory plot using Plotly."""
    sample = dataset[sample_idx]
    history = sample['history'].numpy() if isinstance(sample['history'], torch.Tensor) else sample['history']
    future = sample['future'].numpy() if isinstance(sample['future'], torch.Tensor) else sample['future']
    
    # Create time axis
    history_time = np.arange(len(history)) / dataset.config['data']['sample_rate']
    future_time = np.arange(len(future)) / dataset.config['data']['sample_rate'] + history_time[-1]
    
    # Create 3D plot
    fig = go.Figure()
    
    # Add history trajectory
    fig.add_trace(go.Scatter3d(
        x=history[:, 0],
        y=history[:, 1],
        z=history_time,
        mode='lines+markers',
        name='History',
        line=dict(color='blue', width=4),
        marker=dict(size=3)
    ))
    
    # Add future trajectory
    fig.add_trace(go.Scatter3d(
        x=future[:, 0],
        y=future[:, 1],
        z=future_time,
        mode='lines+markers',
        name='Future',
        line=dict(color='green', width=4),
        marker=dict(size=3)
    ))
    
    # Add start and end markers
    fig.add_trace(go.Scatter3d(
        x=[history[0, 0]],
        y=[history[0, 1]],
        z=[history_time[0]],
        mode='markers',
        name='Start',
        marker=dict(color='red', size=10, symbol='diamond')
    ))
    
    fig.add_trace(go.Scatter3d(
        x=[future[-1, 0]],
        y=[future[-1, 1]],
        z=[future_time[-1]],
        mode='markers',
        name='End',
        marker=dict(color='darkred', size=10, symbol='square')
    ))
    
    # Update layout
    fig.update_layout(
        title=f'3D Trajectory Visualization (Sample {sample_idx})',
        scene=dict(
            xaxis_title='X Position (m)',
            yaxis_title='Y Position (m)',
            zaxis_title='Time (s)',
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.5)
            )
        ),
        height=700,
        showlegend=True
    )
    
    return fig

# Create 3D visualization
fig_3d = create_3d_trajectory_plot(dataset, sample_idx=0)
fig_3d.show()

## 10. Animated Trajectory Visualization

In [ ]:
def create_trajectory_animation(dataset, sample_idx, fps=10):
    """Create an animated visualization of vehicle trajectory."""
    sample = dataset[sample_idx]
    history = sample['history'].numpy() if isinstance(sample['history'], torch.Tensor) else sample['history']
    future = sample['future'].numpy() if isinstance(sample['future'], torch.Tensor) else sample['future']
    
    # Combine history and future
    full_trajectory = np.vstack([history, future])
    total_frames = len(full_trajectory)
    
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Set axis limits
    margin = 10
    ax.set_xlim(full_trajectory[:, 0].min() - margin, full_trajectory[:, 0].max() + margin)
    ax.set_ylim(full_trajectory[:, 1].min() - margin, full_trajectory[:, 1].max() + margin)
    ax.set_xlabel('X Position (meters)')
    ax.set_ylabel('Y Position (meters)')
    ax.set_title(f'Animated Vehicle Trajectory (Sample {sample_idx})')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')
    
    # Initialize lines and points
    history_line, = ax.plot([], [], 'b-', linewidth=2, label='History', alpha=0.7)
    future_line, = ax.plot([], [], 'g-', linewidth=2, label='Future', alpha=0.7)
    current_point, = ax.plot([], [], 'ro', markersize=10, label='Current Position')
    
    # Add time text
    time_text = ax.text(0.02, 0.98, '', transform=ax.transAxes, 
                       verticalalignment='top', fontsize=12)
    
    ax.legend(loc='upper right')
    
    def init():
        history_line.set_data([], [])
        future_line.set_data([], [])
        current_point.set_data([], [])
        time_text.set_text('')
        return history_line, future_line, current_point, time_text
    
    def animate(frame):
        # Update history line (up to current frame, but max history length)
        if frame < len(history):
            history_line.set_data(full_trajectory[:frame+1, 0], full_trajectory[:frame+1, 1])
            future_line.set_data([], [])
        else:
            history_line.set_data(history[:, 0], history[:, 1])
            future_idx = frame - len(history) + 1
            future_line.set_data(future[:future_idx, 0], future[:future_idx, 1])
        
        # Update current position
        current_point.set_data([full_trajectory[frame, 0]], [full_trajectory[frame, 1]])
        
        # Update time
        current_time = frame / dataset.config['data']['sample_rate']
        phase = 'History' if frame < len(history) else 'Future'
        time_text.set_text(f'Time: {current_time:.1f}s ({phase})')
        
        return history_line, future_line, current_point, time_text
    
    anim = FuncAnimation(fig, animate, init_func=init, frames=total_frames,
                        interval=1000/fps, blit=True, repeat=True)
    
    plt.close()  # Prevent static display
    return anim

# Create animation
print("🎬 Creating trajectory animation...")
anim = create_trajectory_animation(dataset, sample_idx=0)
HTML(anim.to_jshtml())

## 11. Model Prediction Visualization (Optional)

In [ ]:
def visualize_model_predictions(model, dataset, sample_idx, device='cpu'):
    """Visualize model predictions vs ground truth."""
    model.eval()
    
    # Get sample
    sample = dataset[sample_idx]
    history = sample['history'].unsqueeze(0).to(device)
    social_tensor = sample['social_tensor'].unsqueeze(0).to(device)
    future_gt = sample['future'].numpy()
    
    # Get model predictions
    with torch.no_grad():
        predictions, mode_probs = model(history, social_tensor)
    
    predictions = predictions.squeeze(0).cpu().numpy()  # [future_len, num_modes, 2]
    mode_probs = mode_probs.squeeze(0).cpu().numpy()  # [num_modes]
    history = history.squeeze(0).cpu().numpy()
    
    # Create visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    
    # Plot trajectories
    ax1.plot(history[:, 0], history[:, 1], 'b-', linewidth=3, 
            label='History', alpha=0.8)
    ax1.plot(future_gt[:, 0], future_gt[:, 1], 'g-', linewidth=3,
            label='Ground Truth', alpha=0.8)
    
    # Plot all predicted modes
    colors = plt.cm.Reds(np.linspace(0.3, 0.9, predictions.shape[1]))
    for mode_idx in range(predictions.shape[1]):
        ax1.plot(predictions[:, mode_idx, 0], predictions[:, mode_idx, 1],
                '--', color=colors[mode_idx], linewidth=2,
                label=f'Mode {mode_idx+1} (p={mode_probs[mode_idx]:.2f})',
                alpha=0.7)
    
    ax1.set_xlabel('X Position (meters)')
    ax1.set_ylabel('Y Position (meters)')
    ax1.set_title('Model Predictions vs Ground Truth')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_aspect('equal')
    
    # Plot mode probabilities
    modes = np.arange(len(mode_probs))
    ax2.bar(modes, mode_probs, color=colors)
    ax2.set_xlabel('Mode')
    ax2.set_ylabel('Probability')
    ax2.set_title('Mode Probabilities')
    ax2.set_xticks(modes)
    ax2.set_xticklabels([f'Mode {i+1}' for i in modes])
    ax2.set_ylim([0, 1])
    ax2.grid(True, alpha=0.3, axis='y')
    
    # Add probability values on bars
    for i, (mode, prob) in enumerate(zip(modes, mode_probs)):
        ax2.text(mode, prob + 0.02, f'{prob:.2f}', ha='center', fontweight='bold')
    
    plt.suptitle(f'Sample {sample_idx}: Model Predictions', fontsize=16, fontweight='bold')
    plt.tight_layout()
    return fig

# Note: This cell requires a trained model
# Uncomment and run if you have a trained model available

# # Load model
# model_path = '../results/csp_waymo/final_model/pytorch_model.bin'
# if os.path.exists(model_path):
#     model = ConvSocialPoolingModel(config)
#     model.load_state_dict(torch.load(model_path, map_location='cpu'))
#     model.eval()
#     
#     # Visualize predictions
#     fig = visualize_model_predictions(model, dataset, sample_idx=0)
#     plt.show()
# else:
#     print("No trained model found. Train the model first to visualize predictions.")

## 12. Batch Visualization Export

In [ ]:
def export_batch_visualizations(dataset, output_dir='visualization_outputs', num_samples=10):
    """Export multiple trajectory visualizations to files."""
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    print(f"📁 Exporting visualizations to {output_path}")
    
    for i in tqdm(range(min(num_samples, len(dataset))), desc="Exporting"):
        sample = dataset[i]
        
        # Create figure with subplots
        fig, axes = plt.subplots(1, 2, figsize=(20, 10))
        
        # Plot trajectory
        plot_trajectory_2d(sample['history'], sample['future'],
                         title=f"Sample {i}: Trajectory",
                         ax=axes[0])
        
        # Plot social tensor
        visualize_social_tensor(sample['social_tensor'], ax=axes[1])
        
        plt.suptitle(f'Waymo Dataset - Sample {i}', fontsize=16, fontweight='bold')
        plt.tight_layout()
        
        # Save figure
        fig.savefig(output_path / f'sample_{i:04d}.png', dpi=100, bbox_inches='tight')
        plt.close(fig)
    
    print(f"✅ Exported {num_samples} visualizations to {output_path}")

# Export visualizations
# export_batch_visualizations(dataset, num_samples=5)

## Summary

This notebook provides comprehensive tools for visualizing and analyzing the Waymo Open Dataset:

1. **Basic trajectory visualization** - 2D plots of vehicle paths
2. **Social context visualization** - Grid-based representation of neighboring vehicles
3. **Multi-vehicle scenes** - Visualize interactions between multiple vehicles
4. **Statistical analysis** - Distributions of velocities, accelerations, and trajectories
5. **Interactive exploration** - Widgets to browse through different samples
6. **3D visualization** - Time-space trajectory representations
7. **Animations** - Dynamic visualization of vehicle movement
8. **Model predictions** - Compare model outputs with ground truth (when available)

Use these tools to:
- Understand the dataset structure
- Debug data loading and preprocessing
- Analyze driving patterns and behaviors
- Evaluate model performance
- Create presentation materials